# 전체 파일 중에서 경은님 작업한 라인넘버 제외하고 나머지만 뽑도록 하는 코드<br>
#### input : 누적 작업 데이터셋<br>
#### output : 전체 데이터셋 - 누적 작업 데이터셋<br>

In [2]:
import json
import csv
import pandas as pd
import os
from os import walk
import re
import datetime
from tqdm.notebook import tnrange

In [7]:
input_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/안합친거'
database_repo = '/Users/jaytsol/Downloads'

output_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/남은 작업'

In [3]:
# 해당 경로의 파일리스트 추출
file_list = []
for (dirpath, dirnames, filenames) in walk(input_repo):
    file_list.extend(filenames)
    break

# 해당 경로의 csv파일의 경로+파일명 리스트 추출
csv_list = []
for i in range(len(file_list)):
    filename, file_extension = os.path.splitext(f'{input_repo}/{file_list[i]}')

    if file_extension == '.csv':
        csv_list.append(filename+file_extension)

In [5]:
# 작업 진행 정도 체크
processed = pd.concat([pd.read_csv(f) for f in csv_list])    # 라인넘버 기준 안합친파일 합치기
processed_lineNumber_list = list(processed['lineNumber'])    # 작업한 lineNumber list

database = pd.read_csv(f'{database_repo}/type2_input_1.csv')    # 누적 데이터베이스 read
database_lineNumber_list = list(database['lineNumber'])    # 누적 데이터베이스의 lineNumber List

print('전체 38451개에서 태깅된 '+str(len(list(database['lineNumber'])))+'개 중 '+str(len(list(processed['lineNumber'])))+'개의 검수를 마쳤습니다.')

전체 38451개에서 태깅된 18114개 중 9373개의 검수를 마쳤습니다.


In [6]:
# 작업한 리스트 : processed_lineNumber_list
# 누적 데이터베이스 리스트 : database_lineNumber_list

In [9]:
processed_lineNumber_set = set(processed['lineNumber'])    # 작업 완료된 문장
database_lineNumber_set = set(database['lineNumber'])    # 전체 데이터베이스

In [10]:
Todo_Task_lineNumber_list = list(database_lineNumber_set.difference(processed_lineNumber_set))    # database - processed.. 즉 아직 작업하지 않은 문장들을 추출

In [13]:
COLUMNS = ['lineNumber', 'originalText', 'transcription']
Todo_Task = pd.DataFrame(columns=COLUMNS)

In [14]:
for i in tnrange(len(Todo_Task_lineNumber_list)):
    for j in range(len(database)):
        if Todo_Task_lineNumber_list[i]==database['lineNumber'][j]:
            Todo_Task = Todo_Task.append({'lineNumber':database['lineNumber'][j], 'originalText':database['originalText'][j], 'transcription':database['transcription'][j]},  ignore_index=True)
        else:
            pass

  0%|          | 0/8742 [00:00<?, ?it/s]

In [21]:
Todo_Task = Todo_Task.sort_values(by=['lineNumber'])    # 작업물 sort by lineNumber

In [12]:
year = str(datetime.datetime.now().year)[2:]
month = str(datetime.datetime.now().month)
day = str(datetime.datetime.now().day)
hour = str(datetime.datetime.now().hour)
minute = str(datetime.datetime.now().minute)
a = [1]
a = pd.DataFrame(a)
a.to_csv(f'{output_repo}/남은 작업_{year}{month}{day}_{hour}{minute}.csv', index=False, encoding='utf-8-sig')